In [1]:
import sys
import re
import itertools
import gurobipy as gp
%load_ext autoreload
%autoreload 2

In [47]:
# value = "x_"+str(11)
def getSolFromFile(value, filename):
    with open(filename, "r") as fo:
        lines = fo.readlines()
    for line in lines:
        ans = re.split(" +",line) # Regular expression split. Treat consective delimiters as one
        v = ans[0]
        s = ans[1]
        if v==value:
            return( float(s))
    return 0.0

# values = ["x_11", "x_33"]
def getSolsFromFile(values, filename):
    """
    values = ["x_11", "x_33"]
    ans = getSolsFromFile(values, "dat/dbgSol.sol")
    print(ans)
    
    The above code fetches the value of variables x_11 and x_33 
    in a gurobi or a scip written solution file dat/dbgSol.sol
    and returns the values in a dictionary
    """    
    ans = {k:0.0 for k in values}
    with open(filename, "r") as fo:
        lines = fo.readlines()
    for line in lines:
        ind = re.split(" +",line) # Regular expression split. Treat consective delimiters as one
        v = ind[0]
        s = ind[1]
        if v in values:
            print(v)
            ans[v] = float(s)
    return ans
def getParLocVar(filename):
    """
    Get the set of parameters, variable locations in the lcp model and values
    from a given file written using EPEC::appendSolution4XL
    If the file has multiple solution value rows, the first of them is chosen
    """
    with open(filename, "r") as fo:
        line1 = fo.readline().split(" ")
        line2 = fo.readline().split(" ")
        line3 = fo.readline().split(" ")

    param = dict()
    var = dict()
    locs = dict()
    for w1, w2, w3 in zip(line1, line2, line3):
        if w2.strip() == '':break
        key = w1.split("_")
        if len(key)>1: key = tuple(key)
        else: key = key[0]
        if float(w2) == -1: # i.e., a Parameter
            param[key] = float(w3)
        else:
            locs[key] = int(w2)
            var[key] =  float(w3)
    return param, locs, var

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def formatIt(a,b = ""):
    if is_number(b):
        return "{:>30}: {:>8}\n".format(a,round(b,2))
    else:
        return "{:>30}: {:>8}\n".format(a,b)
    
def getGurVar(model, locationDict, variable):
    return model.getVarByName("x_"+str(locationDict[variable]))

In [48]:
class EPECio:
    def __init__(self, countries, producers, dirty, clean, n_Scen=1):
        """
        countries should be a list of countries
        producers should be a dictionary, where the keys are elements of countries. 
                producers[cc] should contain the set of producers in country cc
        dirty is the set of dirty energy technologies
        clean is the set of clean energy technologies
        """
        self.countries = countries
        self.producers = producers
        self.dirty = dirty
        self.clean = clean
        self.n_Scen = n_Scen
        self.energy = dirty+clean
    def acquireData(self,filename):
        self.param, self.locs, self.var = getParLocVar (filename)
    def writeFoll(self,filename, cc, ff):
        with open(filename,"a") as fo:
            fo.write(cc+"---"+ff+"\n")
            fo.write(formatIt("Carbon Limit",self.var[(cc,ff,"CarbBuy")]))
            fo.write("\nInvestments by "+ff+"\n")
            for gg in ["solar","wind"]:
                fo.write(formatIt("Investment in "+gg,self.var[(cc,ff,"Inv",gg)]))
            fo.write("\n")
            # Scenario wise production
            expProd = {ee:0 for ee in self.energy}
            for xi in range(self.n_Scen):
                if self.n_Scen>1: fo.write("Scenario: "+str(xi+1)+"\n")
                if self.n_Scen>1:fo.write("  Production: "+"\n")
                for ee in self.energy:
                    val = self.var[(cc,ff,"Prod",ee,"xi"+str(xi))]
                    if self.n_Scen>1: fo.write(formatIt(ee+": ",val))
                    expProd[ee] += val/self.n_Scen
                if self.n_Scen>1: fo.write("\n")
            # Expected production
            fo.write(" EXPECTED Production: "+"\n")
            for ee in self.energy:
                fo.write(formatIt(ee+": ",expProd[ee]))
            fo.write(formatIt("Total: ",sum(expProd[ee] for ee in self.energy)))
            fo.write("\n")
            fo.write("-----------------------------------------")
            fo.write("\n")
            return expProd
                
    def writeCoun(self,filename, cc):
        with open(filename,"a") as fo:
            fo.write(cc + ":\n")
            fo.write(formatIt("Carbon Import:",self.var[(cc,"CarbImp")]))
            fo.write(formatIt("Carbon Tax:",self.var[(cc,"CarbTax")]))
        Prod = dict()
        for ff in self.producers[cc]:
            Prod[ff] = self.writeFoll(filename,cc,ff)
        with open(filename,"a") as fo:
            for ee in self.energy:
                fo.write(formatIt("Country total "+ee+": ",sum(Prod[ff][ee] for ff in self.producers[cc])))
            fo.write(formatIt("Country Total Prodn: ",sum(Prod[ff][ee] for ff in self.producers[cc]
                                                          for ee in self.energy
                                                         )))

    def writeAll(self,filename):
        with open(filename,"w") as fo:
            fo.write("###########################################\n")
            carbSuppl = sum( [self.var [(cc, "CarbImp")] for cc in self.countries ])
            fo.write(formatIt("Total Carbon Supplied:", carbSuppl))
            fo.write(formatIt("Carbon Price:", str(self.param["suppInt"]) + " + " + str(self.param["suppSlope"]) + "Q"))
            fo.write(formatIt(" ", self.param["suppInt"] + self.param["suppSlope"]*carbSuppl ))
            fo.write("###########################################\n")
        for cc in self.countries:
            self.writeCoun(filename, cc)
            with open(filename,"a") as fo:
                fo.write("###########################################\n")
                
        

# Commands

In [70]:
prefix = "Main2"
argv = ["readWrite", "../dat/"+prefix+"solLog.txt", "../dat/"+prefix+"lcpmodel.lp", "../dat/"+prefix+"rRdblOut.txt"]

In [71]:
countries = ["c1","c2"]
producers = {"c1":["F1","F2"],"c2":["F3","F4"]}
dirty = ["coal", "gas"]
clean = ["solar", "wind"]
epec = EPECio(countries, producers, dirty, clean)
epec.acquireData(argv[1])

In [72]:
M = gp.read(argv[2])
expr = M.getObjective()
expr += gp.quicksum([getGurVar(M, epec.locs, (cc,"CarbImp")) for cc in countries])*0.1
M.setObjective(expr)
M.optimize()
M.write("../dat/dbgSol.sol")

Read LP format model from file ../dat/Main2lcpmodel.lp
Reading time = 0.01 seconds
: 1704 rows, 3408 columns, 6098 nonzeros
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 1704 rows, 3408 columns and 6098 nonzeros
Model fingerprint: 0xae0e7dc9
Model has 1704 general constraints
Variable types: 1704 continuous, 1704 integer (1704 binary)
Coefficient statistics:
  Matrix range     [2e-04, 4e+03]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 1e+00]
Presolve removed 1296 rows and 2786 columns
Presolve time: 0.08s
Presolved: 408 rows, 622 columns, 1738 nonzeros
Presolved model has 127 SOS constraint(s)
Variable types: 490 continuous, 132 integer (132 binary)

Root relaxation: objective 2.847993e+05, 260 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 284799.291    0    2  

In [73]:
newVars = dict()
for kk in epec.locs.keys():
    vv = epec.locs[kk]
    newVars[kk] = getSolFromFile("x_"+str(vv), "../dat/dbgSol.sol")
epec.var = newVars
# epec.writeAll("../dat/temp2.txt")

In [74]:
epec.writeAll(argv[3])

In [75]:
expr

<gurobi.LinExpr: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_10 + x_11 + x_12 + x_13 + 2.0 x_14 + x_15 + x_16 + x_17 + x_18 + x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 + x_27 + x_28 + x_29 + x_30 + x_31 + x_32 + x_33 + x_34 + x_35 + x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 + x_44 + x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 + x_54 + x_55 + x_56 + x_57 + x_58 + x_59 + x_60 + x_61 + x_62 + x_63 + x_64 + x_65 + x_66 + x_67 + x_68 + x_69 + x_70 + x_71 + x_72 + x_73 + x_74 + x_75 + x_76 + x_77 + x_78 + x_79 + x_80 + x_81 + x_82 + x_83 + x_84 + x_85 + x_86 + x_87 + x_88 + x_89 + x_90 + x_91 + x_92 + x_93 + x_94 + x_95 + x_96 + x_97 + x_98 + x_99 + x_100 + x_101 + x_102 + x_103 + x_104 + x_105 + x_106 + x_107 + x_108 + x_109 + x_110 + x_111 + x_112 + x_113 + x_114 + x_115 + x_116 + x_117 + x_118 + x_119 + x_120 + x_121 + x_122 + x_123 + x_124 + x_125 + x_126 + x_127 + x_128 + x_129 + x_130 + x_131 + x_132 + x_133 + x_134 + x_135 + x